# LightSB I2I Implementation on BDD100K using the Shipped ALAE Encoder

#### Make sure the kernel is YOLOPX_LIGHTSB!

This notebook attempts a passthrough of the 

# Config
Setup YOLOPX path and namespace, setup LightSB path. Prepare everything we need to procure the data.

In [1]:
# Appends the YOLOPX directory to the path, so we can import the BDD100K dataloader
import os, sys

path = os.getcwd()
BASE_DIR = os.path.dirname(os.path.dirname(path))
print(BASE_DIR)
assert(os.path.basename(BASE_DIR) == "LightSB_YOLO")
YOLOPX_DIR = os.path.join(BASE_DIR, "", "models\YOLOPX")

sys.path.append(YOLOPX_DIR)
print(YOLOPX_DIR)

c:\Users\joche\Development\Github\LightSB_YOLO
c:\Users\joche\Development\Github\LightSB_YOLO\models\YOLOPX


In [2]:
from lib.utils import DataLoaderX

c:\Users\joche\anaconda3\envs\yolopx_lightsb\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
LIGHTSB_DIR = os.path.join(BASE_DIR, "", "LightSB")
sys.path.append(LIGHTSB_DIR)
print(LIGHTSB_DIR)

c:\Users\joche\Development\Github\LightSB_YOLO\LightSB


# Setup Namespace

In [ ]:
from argparse import Namespace
from lib.config import cfg, update_config

args = Namespace()
args.iou_thres = 0.6
args.conf_thres = 0.001
args.weights = "../weights/epoch-195.pth"
args.modelDir = ''
args.logDir = 'runs/'

# annotations directory setup
args.da_seg_annotations = os.path.join(BASE_DIR, '', 'data/da_seg_annotations')
args.det_annotations = os.path.join(BASE_DIR, '', 'data/det_annotations')
args.images = os.path.join(BASE_DIR, '', 'data/images')
args.ll_seg_annotations = os.path.join(BASE_DIR, '', 'data/ll_seg_annotations')
update_config(cfg, args)

# Data
1. Get full training set from YOLOPX. 
2. Use select_data to get separate domains, daytime and nighttime.
3. 

In [32]:
import lib.dataset as dataset
import pickle as pkl

pkl_dir = os.path.join(os.getcwd(), 'pkl_files', 'full_train_dataset.pkl')
skippable = os.path.exists(pkl_dir)
print(pkl_dir)
print("Skippable:", skippable)

full_train_dataset = dataset.BddDataset(
    cfg=cfg,
    is_train=True,
    inputsize=cfg.MODEL.IMAGE_SIZE,
    transform=None, 
    skip=skippable
)

if not skippable:
    with open(pkl_dir, 'wb') as f:
        pkl.dump(full_train_dataset.db, f)

else:
    with open(pkl_dir, 'rb') as f:
        full_train_dataset.db = pkl.load(f)

c:\Users\joche\Development\Github\LightSB_YOLO\scripts\LightSB\pkl_files\full_train_dataset.pkl
Skippable: True


c:\Users\joche\anaconda3\envs\yolopx_lightsb\lib\site-packages\albumentations\core\composition.py:192: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


In [38]:
import copy
print('Full training set samples:', len(full_train_dataset.db))

train_day = copy.copy(full_train_dataset)
train_day.db = train_day.select_data(timeofday='daytime')
print('Daytime samples:', len(train_day.db))

train_night = copy.copy(full_train_dataset)
train_night.db = train_night.select_data(timeofday='night')
print('Night samples:', len(train_night.db))

print('Not included samples:', len(full_train_dataset.db) - len(train_day.db) - len(train_night.db))

Full training set samples: 70000
Daytime samples: 36800
Night samples: 28028
Not included samples: 5172
